## NADILA IMAARAH 3323600015_D4 SAINS DATA TERAPAN A

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Membaca Dataset Titanic

In [13]:
dataset = pd.read_csv("titanic.csv")
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Hold Out Method

**Membagi dataset dengan hold out method**

In [14]:
train_ho, test_ho = train_test_split(dataset, 
                                     test_size=0.3, random_state=42)

**ANALISIS**
disini membagi dataset menjdadi data test dan data train dengan menggunakan metode hold out, dimana 30% dijadikan data tes dan 70% dijadikan data train

kita menggunakan random_state agar urutan pengambilanya sama tidak berubah rubah

In [15]:
train_data_ho = train_ho[['Sex', 'Age', 'Pclass', 'Fare']].copy()
test_data_ho = test_ho[['Sex', 'Age', 'Pclass', 'Fare']].copy()

**ANALISIS**

setelah membagi data train, di sini saya mengambil beberapa fitur saja yakni sex,age,pclass dan fare dari data train dan data tes yang telah di bagi, saya menggunakan copy agar tidak merusak data aslinya

**Handling missing value**

In [16]:
train_data_ho['Age'] = train_data_ho.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))
test_data_ho['Age'] = test_data_ho.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))

**ANALISIS**

Disini saya mengisi missing value pada kolom age dengan rata-rata age berdasarkan kelas masing-masing

In [17]:
# Mengambil kolom 'Survived' sebagai target
label_train_ho = train_ho[['Survived']]
label_test_ho = test_ho[['Survived']]

**ANALISIS**

jadi karena study case ini klasifikasi maka fitur target harus dipisahkan dulu agar model bisa belajar untuk bisa melakukan prediksi

**ENCODING**

In [ ]:
train_data_ho["Sex"] = train_data_ho["Sex"].map({"male": 0, "female": 1})
test_data_ho["Sex"] = test_data_ho["Sex"].map({"male": 0, "female": 1})


,Sex,Age,Pclass,Fare
445,NaN,4.000000,1,81.8583
650,NaN,25.029098,3,7.8958
172,NaN,1.000000,3,11.1333
450,NaN,36.000000,2,27.7500
314,NaN,43.000000,2,26.2500
...,...,...,...,...
106,NaN,21.000000,3,7.6500
270,NaN,37.893443,1,31.0000
860,NaN,41.000000,3,14.1083
435,NaN,14.000000,1,120.0000


**ANALISIS**

selanjutnya melakukan encoding, karena metode klasifikasi tidak bisa langsung mengolah data teks jadi perlu dilakukan encoding, yang awalnya berupa teks menjadi numerik

**Normalisasi dengan MinMax**

In [19]:

scaler = MinMaxScaler()
train_ho_scaled = scaler.fit_transform(train_data_ho)
test_ho_scaled = scaler.transform(test_data_ho)

**ANALISIS**

Selanjutnya dilakukan normalisasi dengan menggunakan minmax agar skala setiap variabel berada rentang 0 hingga 1 

**Mengklasifikasi dengan KNN**

In [20]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_ho_scaled, label_train_ho)
y_pred_ho = knn.predict(test_ho_scaled)
error_ratio_ho = 1 - accuracy_score(label_test_ho, y_pred_ho)
print(f"Error Ratio Hold-Out: {error_ratio_ho}")

Error Ratio Hold-Out: 0.18656716417910446


C:\Users\nimaa\AppData\Roaming\Python\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


**ANALISIS**

Selenjutnya membuat model knn dengan n sebanyak 3 yang artinya model akan memprediksi label data baru dengan melihat 3 tetangga terdekat, setelah itu melatih model knn dengan data train yang telah di scaled dan data label

Kemudian menghitung akurasi antara prediksi dan label asli dan setelah itu mengihitung nilai error rasio yang didapatkan dari 1 di kurangi  akurasi

# K-Fold Method

In [21]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
error_ratios_kfold = []

for train_idx, test_idx in kf.split(dataset):
    # Membagi dataset menjadi train dan test
    train_kf, test_kf = dataset.iloc[train_idx], dataset.iloc[test_idx]

    # Mengambil fitur dan mengisi missing value Age dengan rata-rata berdasarkan Pclass
    train_data_kf = train_kf[['Sex', 'Age', 'Pclass', 'Fare']].copy()
    test_data_kf = test_kf[['Sex', 'Age', 'Pclass', 'Fare']].copy()

    train_data_kf['Age'] = train_data_kf.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))
    test_data_kf['Age'] = test_data_kf.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))

    # Mengambil kolom 'Survived' sebagai label
    label_train_kf = train_kf[['Survived']]
    label_test_kf = test_kf[['Survived']]

    #Encoding pada kolom sex
    train_data_kf["Sex"] = train_data_kf["Sex"].map({"male": 0, "female": 1})
    test_data_kf["Sex"] = test_data_kf["Sex"].map({"male": 0, "female": 1})
    
    # Normalisasi Min-Max
    scaler = MinMaxScaler()
    train_kf_scaled = scaler.fit_transform(train_data_kf)
    test_kf_scaled = scaler.transform(test_data_kf)

    # Klasifikasi K-NN dan menghitung error ratio
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(train_kf_scaled, label_train_kf.values.ravel())
    y_pred_kf = knn.predict(test_kf_scaled)
    error_ratio_kf = 1 - accuracy_score(label_test_kf, y_pred_kf)
    error_ratios_kfold.append(error_ratio_kf)

# Menghitung rata-rata error ratio
mean_error_ratio_kfold = np.mean(error_ratios_kfold)
print(f"Error Ratio K-Fold (k=10): {mean_error_ratio_kfold}")

Error Ratio K-Fold (k=10): 0.19189762796504367


**Analisis**
tahapanya sama seperti sebelumnya yakni, Namun di sini metode untuk membagi datasetnya berbeda yakni dengan metode kfold. Kfold akan membagi dataset menjadi fold atau k bagian yang salin tidak tumpeng tindih. Disini dataset akan dibagi menjadi 10 bagian atau fold, setiap fold terdiri dari data yang acak. Pada iterasi pertama fold pertama akan digunakan untuk training dan fold 2 hingga 10 akan dijadikan testing, kemudian pada iterasi kedua fold kedua digunakan untuk training dan fold lainya dijadikan testing. proses ini diulang hingga semua fold telah digunakna sebagai data uji

# Leave-one-out Cross Validation

In [22]:
loo = LeaveOneOut()
error_ratios_loo = []

for train_idx, test_idx in loo.split(dataset):
    # Membagi dataset menjadi train dan test
    train_loo, test_loo = dataset.iloc[train_idx], dataset.iloc[test_idx]

    # Mengambil fitur dan mengisi missing value Age dengan rata-rata berdasarkan Pclass
    train_data_loo = train_loo[['Sex', 'Age', 'Pclass', 'Fare']].copy()
    test_data_loo = test_loo[['Sex', 'Age', 'Pclass', 'Fare']].copy()

    train_data_loo['Age'] = train_data_loo.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))
    test_data_loo['Age'] = test_data_loo['Age'].fillna(train_data_loo['Age'].mean())

    # Mengambil kolom 'Survived' sebagai label
    label_train_loo = train_loo[['Survived']]
    label_test_loo = test_loo[['Survived']]

    #Encoding pada kolom sex
    train_data_loo["Sex"] = train_data_loo["Sex"].map({"male": 0, "female": 1})
    test_data_loo["Sex"] = test_data_loo["Sex"].map({"male": 0, "female": 1})
    
     # Normalisasi Min-Max
    scaler = MinMaxScaler()
    train_loo_scaled = scaler.fit_transform(train_data_loo)
    test_loo_scaled = scaler.transform(test_data_loo)

    # Klasifikasi K-NN dan menghitung error ratio
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(train_loo_scaled, label_train_loo.values.ravel())
    y_pred_loo = knn.predict(test_loo_scaled)
    error_ratio_loo = 1 - accuracy_score(label_test_loo, y_pred_loo)
    error_ratios_loo.append(error_ratio_loo)

# Menghitung rata-rata error ratio
mean_error_ratio_loo = np.mean(error_ratios_loo)
print(f"Error Ratio LOO: {mean_error_ratio_loo}")

Error Ratio LOO: 0.19079685746352412


**ANALISIS**
Namun di sini metode untuk membagi datasetnya berbeda yakni dengan metode leave one out cross validation.Metode ini bekerja dengan cara membagi dataset dengan setiap data dalam dataset digunakan 1 kalli sebagai data uji dan data lainya menjadi data testing, missal kita memiliki 100 data, maka LOO-CV akan melakukan 100 iterasi, di mana pada setiap iterasi, satu data point digunakan sebagai data uji, dan 99 data lainnya digunakan untuk pelatihan. 

setelah membagi dataset langkahnya sama seperti tahapan sebeelumnya  yakni mengisi missing value pada data age berdasarkan kelas masing-masing, kemudian memisahkan target, lalu melakukan encoding pada data yang awalnya string menjadi numerik dan juga melakukan normalisasi dengan metode yang sama yakni minmax
